[download this notebook here](https://github.com/HumanCompatibleAI/imitation/blob/master/docs/tutorials/5_train_preference_comparisons.ipynb)
# We want to compare pairwise comparison with groupwise comparison

We will use synthetic feedback based on the true reward function of the reacher environment to evaluate and compare pairwise comparison and pairwise group comparison.

To set up the preference comparisons algorithm, we first need to set up a lot of its internals beforehand:

In [1]:
import random
from imitation.algorithms import preference_comparisons
from imitation.rewards.reward_nets import BasicRewardNet, RewardEnsemble
from imitation.util.networks import RunningNorm
from imitation.util.util import make_vec_env
from imitation.policies.base import FeedForward32Policy, NormalizeFeaturesExtractor
from imitation.regularization.regularizers import LpRegularizer
from imitation.regularization.updaters import IntervalParamScaler
import gymnasium as gym
from stable_baselines3 import PPO
import numpy as np
from imitation.util import logger
import stable_baselines3.common.logger as sb_logger


rng = np.random.default_rng(0)
def intantiate_and_train(pairwise):
    venv = make_vec_env("Reacher-v4", rng=rng, render_mode='rgb_array', n_envs=8)

    reward_net_members = [BasicRewardNet(venv.observation_space, venv.action_space, normalize_input_layer=RunningNorm) for _ in range(5)]
    reward_net = RewardEnsemble(venv.observation_space, venv.action_space, reward_net_members)

    preference_model = preference_comparisons.PreferenceModel(reward_net)
    # reward_trainer = preference_comparisons.BasicRewardTrainer(
    #     preference_model=preference_model,
    #     loss=preference_comparisons.CrossEntropyRewardLoss(),
    #     epochs=3,
    #     rng=rng,
    # )


    # Create a lambda updater
    scaling_factor = 0.1
    tolerable_interval = (0.9, 1.1) 
    lambda_updater = IntervalParamScaler(scaling_factor, tolerable_interval)
    # Create a RegularizerFactory
    regularizer_factory = LpRegularizer.create(initial_lambda=0.1, lambda_updater=lambda_updater, p=2, val_split=0.1)

    reward_trainer = preference_comparisons.EnsembleTrainer(
        preference_model,
        loss=preference_comparisons.CrossEntropyRewardLoss(),
        rng=rng,
        epochs=5,
        batch_size = 4,
        minibatch_size = 2,
        # lr: float = 1e-3,
        # custom_logger: Optional[imit_logger.HierarchicalLogger] = None,
        regularizer_factory = regularizer_factory,
    )
    if pairwise:
        base_fragmenter = preference_comparisons.RandomFragmenter(
            warning_threshold=0,
            rng=rng,
        )
        fragmenter = preference_comparisons.ActiveSelectionFragmenter(
                preference_model,
                base_fragmenter,
                2.0,
        )
        gatherer = preference_comparisons.SyntheticGatherer(rng=rng)
    else:
        fragmenter = preference_comparisons.AbsoluteUncertaintyFragmenter(
            preference_model,
            2.0,
            rng=rng,
        )
        gatherer = preference_comparisons.SyntheticGathererForGroupComparisons(rng=rng)
    # Several hyperparameters (reward_epochs, ppo_clip_range, ppo_ent_coef,
    # ppo_gae_lambda, ppo_n_epochs, discount_factor, use_sde, sde_sample_freq,
    # ppo_lr, exploration_frac, num_iterations, initial_comparison_frac,
    # initial_epoch_multiplier, query_schedule) used in this example have been
    # approximately fine-tuned to reach a reasonable level of performance.
    agent = PPO(
        policy=FeedForward32Policy,
        policy_kwargs=dict(
            features_extractor_class=NormalizeFeaturesExtractor,
            features_extractor_kwargs=dict(normalize_class=RunningNorm),
        ),
        env=venv,
        seed=0,
        n_steps=2048 // venv.num_envs,
        batch_size=64,
        ent_coef=0.01,
        learning_rate=2e-3,
        clip_range=0.1,
        gae_lambda=0.95,
        gamma=0.97,
        n_epochs=10,
        tensorboard_log="tensorboard_logs/",
    )

    trajectory_generator = preference_comparisons.AgentTrainer(
        algorithm=agent,
        reward_fn=reward_net,
        venv=venv,
        rng=rng,
        exploration_frac=0.05,
    )

    default_logger = sb_logger.Logger(folder='/logs', output_formats='stdout,log,csv,tensorboard')
    custom_logger = logger.HierarchicalLogger(default_logger=default_logger)

    pref_comparisons = preference_comparisons.PreferenceComparisons(
        trajectory_generator,
        reward_net,
        num_iterations=5,  # Set to 60 for better performance
        fragmenter=fragmenter,
        preference_gatherer=gatherer,
        reward_trainer=reward_trainer,
        fragment_length=50,
        transition_oversampling=1,
        initial_comparison_frac=0.1,
        allow_variable_horizon=False,
        initial_epoch_multiplier=4,
        query_schedule="hyperbolic",
        custom_logger=custom_logger,
    )

    return pref_comparisons.train(
        total_timesteps=100_000,
        total_comparisons=1000,
    )

pygame 2.5.2 (SDL 2.28.2, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


Classical pairwise comparison (baseline):

In [3]:

pairwise_comparison_result = intantiate_and_train(True)    

print(pairwise_comparison_result)

Query schedule: [100, 254, 204, 170, 145, 127]
Fewer transitions available than needed for desired number of fragment pairs. Some transitions will appear multiple times.


Training reward model:   0%|          | 0/20 [00:00<?, ?it/s]

Training reward model:   0%|          | 0/20 [00:00<?, ?it/s]

Training reward model:   0%|          | 0/20 [00:00<?, ?it/s]

Training reward model:   0%|          | 0/20 [00:00<?, ?it/s]

...

Training reward model:   0%|          | 0/5 [00:00<?, ?it/s]

Training reward model:   0%|          | 0/5 [00:00<?, ?it/s]

Training reward model:   0%|          | 0/5 [00:00<?, ?it/s]

------------------------------------------------------
| raw/                                 |             |
|    agent/rollout/ep_len_mean         | 50          |
|    agent/rollout/ep_rew_mean         | -16.4       |
|    agent/rollout/ep_rew_wrapped_mean | -7.93       |
|    agent/time/fps                    | 2818        |
|    agent/time/iterations             | 1           |
|    agent/time/time_elapsed           | 0           |
|    agent/time/total_timesteps        | 104448      |
|    agent/train/approx_kl             | 0.017879821 |
|    agent/train/clip_fraction         | 0.51        |
|    agent/train/clip_range            | 0.1         |
|    agent/train/entropy_loss          | 0.0479      |
|    agent/train/explained_variance    | 0.885       |
|    agent/train/learning_rate         | 0.002       |
|    agent/train/loss                  | -0.0095     |
|    agent/train/n_updates             | 500         |
|    agent/train/policy_gradient_loss  | -0.00993    |
|    agent

-----------------------------------------------------
| raw/                                 |            |
|    agent/rollout/ep_len_mean         | 50         |
|    agent/rollout/ep_rew_mean         | -10.7      |
|    agent/rollout/ep_rew_wrapped_mean | -3.39      |
|    agent/time/fps                    | 1161       |
|    agent/time/iterations             | 9          |
|    agent/time/time_elapsed           | 15         |
|    agent/time/total_timesteps        | 120832     |
|    agent/train/approx_kl             | 0.01875526 |
|    agent/train/clip_fraction         | 0.492      |
|    agent/train/clip_range            | 0.1        |
|    agent/train/entropy_loss          | 0.764      |
|    agent/train/explained_variance    | 0.939      |
|    agent/train/learning_rate         | 0.002      |
|    agent/train/loss                  | 0.00136    |
|    agent/train/n_updates             | 580        |
|    agent/train/policy_gradient_loss  | -0.000496  |
|    agent/train/std        

Pairwise Group comparison:

In [2]:
pairwise_group_comparison_result = intantiate_and_train(False)    

print(pairwise_group_comparison_result)

Query schedule: [100, 254, 204, 170, 145, 127]
DBSCAN found 30 clusters. 30 groups were created.
Before cleaning: Group 1 has 3 fragments, Group 2 has 3 fragments
Group 1 has an average reward of -67.74492009480794, Group 2 has an average reward of -64.79341125488281
Groups reward border is -66.26916567484537
Group 1 has 2 fragments, Group 2 has 2 fragments
Before cleaning: Group 1 has 5 fragments, Group 2 has 3 fragments
Group 1 has an average reward of -61.35270538330078, Group 2 has an average reward of -62.25755182902018
Groups reward border is -61.80512860616048
Group 1 has 4 fragments, Group 2 has 2 fragments
Before cleaning: Group 1 has 4 fragments, Group 2 has 7 fragments
Group 1 has an average reward of -61.536930084228516, Group 2 has an average reward of -62.538499559674946
Groups reward border is -62.03771482195173
Group 1 has 3 fragments, Group 2 has 3 fragments
Before cleaning: Group 1 has 5 fragments, Group 2 has 5 fragments
Group 1 has an average reward of -62.154240417

Training reward model:   0%|          | 0/20 [00:00<?, ?it/s]

Training reward model:   0%|          | 0/20 [00:00<?, ?it/s]

Training reward model:   0%|          | 0/20 [00:00<?, ?it/s]

Training reward model:   0%|          | 0/20 [00:00<?, ?it/s]

...

Training reward model:   0%|          | 0/5 [00:00<?, ?it/s]

Training reward model:   0%|          | 0/5 [00:00<?, ?it/s]

Training reward model:   0%|          | 0/5 [00:00<?, ?it/s]

------------------------------------------------------
| raw/                                 |             |
|    agent/rollout/ep_len_mean         | 50          |
|    agent/rollout/ep_rew_mean         | -14.1       |
|    agent/rollout/ep_rew_wrapped_mean | -24.7       |
|    agent/time/fps                    | 2895        |
|    agent/time/iterations             | 1           |
|    agent/time/time_elapsed           | 0           |
|    agent/time/total_timesteps        | 104448      |
|    agent/train/approx_kl             | 0.018814774 |
|    agent/train/clip_fraction         | 0.49        |
|    agent/train/clip_range            | 0.1         |
|    agent/train/entropy_loss          | 0.18        |
|    agent/train/explained_variance    | 0.902       |
|    agent/train/learning_rate         | 0.002       |
|    agent/train/loss                  | 0.0984      |
|    agent/train/n_updates             | 500         |
|    agent/train/policy_gradient_loss  | -0.00627    |
|    agent

------------------------------------------------------
| raw/                                 |             |
|    agent/rollout/ep_len_mean         | 50          |
|    agent/rollout/ep_rew_mean         | -9.73       |
|    agent/rollout/ep_rew_wrapped_mean | -18.2       |
|    agent/time/fps                    | 1247        |
|    agent/time/iterations             | 9           |
|    agent/time/time_elapsed           | 14          |
|    agent/time/total_timesteps        | 120832      |
|    agent/train/approx_kl             | 0.023953147 |
|    agent/train/clip_fraction         | 0.54        |
|    agent/train/clip_range            | 0.1         |
|    agent/train/entropy_loss          | 0.899       |
|    agent/train/explained_variance    | 0.919       |
|    agent/train/learning_rate         | 0.002       |
|    agent/train/loss                  | 0.042       |
|    agent/train/n_updates             | 580         |
|    agent/train/policy_gradient_loss  | -0.00025    |
|    agent

Active selection fragmenter based on random fragmenter:

In [ ]:
pref_comparisons_3.train(
    total_timesteps=50_000,
    total_comparisons=500,
)

After we trained the reward network using the preference comparisons algorithm, we can wrap our environment with that learned reward.

In [5]:
from imitation.rewards.reward_wrapper import RewardVecEnvWrapper

learned_reward_venv = RewardVecEnvWrapper(venv, reward_net.predict_processed)

Next, we train an agent that sees only the shaped, learned reward.

In [6]:
learner = PPO(
    seed=0,
    policy=FeedForward32Policy,
    policy_kwargs=dict(
        features_extractor_class=NormalizeFeaturesExtractor,
        features_extractor_kwargs=dict(normalize_class=RunningNorm),
    ),
    env=learned_reward_venv,
    batch_size=64,
    ent_coef=0.01,
    n_epochs=10,
    n_steps=2048 // learned_reward_venv.num_envs,
    clip_range=0.1,
    gae_lambda=0.95,
    gamma=0.97,
    learning_rate=2e-3,
)
learner.learn(100_000)  # Note: set to 100_000 to train a proficient expert

Then we can evaluate it using the original reward.

In [7]:
from stable_baselines3.common.evaluation import evaluate_policy

n_eval_episodes = 100
reward_mean, reward_std = evaluate_policy(learner.policy, venv, n_eval_episodes)
reward_stderr = reward_std / np.sqrt(n_eval_episodes)
print(f"Reward: {reward_mean:.0f} +/- {reward_stderr:.0f}")

Reward: -11 +/- 0


In [8]:
learner.save('imitation_ppo')

In [9]:
from gymnasium.wrappers import RecordVideo

# Create the environment
env = gym.make("Reacher-v4", render_mode='rgb_array')
env = RecordVideo(env, './evaluation_videos', name_prefix="reacher", episode_trigger=lambda x: x % 1 == 0) 

# Run the model in the environment
obs, info = env.reset()
for _ in range(1000):
        action, _states = learner.predict(obs, deterministic=True)
        obs, reward, _ ,done, info = env.step(action)
        if done:
            obs, info = env.reset()
            

env.close()

/venv/lib/python3.8/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /imitation/docs/tutorials/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-0.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-0.mp4
Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-1.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-1.mp4
Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-2.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-2.mp4
Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-3.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-3.mp4



...

Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-16.mp4
Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-17.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-17.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-17.mp4
Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-18.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-18.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-18.mp4
Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-19.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-19.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-19.mp4
Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-20.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-20.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-20.mp4
